Import Librarys

In [23]:
import pandas as pd
import numpy as np
import scikitplot as skplt

Load The CSVs to DFs

In [24]:
penguindf = pd.read_csv('penguins.csv')
abalonedf = pd.read_csv('abalone.csv')

In [29]:
penguin_columns_to_encode = ['species','island','sex']
penguindf_encoded = pd.get_dummies(penguindf, columns=penguin_columns_to_encode, drop_first=True)
print(penguindf_encoded)


     culmen_length_mm  culmen_depth_mm  flipper_length_mm  body_mass_g  \
0                39.1             18.7                181         3750   
1                39.5             17.4                186         3800   
2                40.3             18.0                195         3250   
3                36.7             19.3                193         3450   
4                39.3             20.6                190         3650   
..                ...              ...                ...          ...   
328              47.2             13.7                214         4925   
329              46.8             14.3                215         4850   
330              50.4             15.7                222         5750   
331              45.2             14.8                212         5200   
332              49.9             16.1                213         5400   

     species_Chinstrap  species_Gentoo  island_Dream  island_Torgersen  \
0                    0               